# Applied Data Science Capstone-Week 3 Peer graded Assignment

## In this notebook I will be segmenting Toronto neighborhoods. 

The first thing to do is to import the necessary libraries in order this notebook to work.

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

The next step is to get our information from the source i.e. https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  , to do that we will use the reqeuests library.

In [2]:
toronto_html=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(toronto_html,'lxml')
#print(soup.prettify())
table = soup.table#find('table', class_='wikitable sortable')
#print(table.prettify())

Here we have created a soup object that contains the text of the html page. Then, we extract the table class from the soup object. From this, we can identify the table rows and columns as follows:

In [12]:
column_names=['PostalCode','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names)
for row in table.find_all('tr'):
    #print(row.prettify())
    row_text=row.text
    column=row_text.split('\n')
    PostalCode=column[1]
    Borough=column[2]
    Neighborhood=column[3]
    df=df.append({'PostalCode':PostalCode,'Borough':Borough,'Neighborhood':Neighborhood}, ignore_index=True)

The columns text is appended to the created dataframe 'df' in each iteration. This gives us the following:

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


As we can see, we need to drop the first row, as we have created our own column names.

In [5]:
df.drop(0, inplace=True)
df.reset_index(drop=True,inplace=True)

The next step is to drop the 'Not assigned' values in the 'Borough' column. To do so, we first replace the 'Not assigned' to np.nan and then used the method .dropna in our data frame.

In [6]:
df.replace('Not assigned', np.nan, inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
df.dropna(subset=['Borough'],axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Lastly, we can manually replace the NaN value in the 'Neighborhood' column:

In [8]:
df['Neighborhood'].replace(np.nan, 'Queen\'s Park',inplace=True)

The last step is to group the postal codes and combine the Neighborhood names into the same row.

In [10]:
df=df.groupby(['PostalCode','Borough'], as_index=False,sort = False).agg(lambda x: ','.join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df.shape

(103, 3)